# Version  
** 1.2 **  
1.2 Support for EXPLORE_MODE, allowing for succinct or full logging of data  
1.1 Added plot support  
1.0 1st release after review.  




This is just an increasing number. If you make a change increase the number - major.minor  


Develop a program in Python using the Keras Neural Network to implement a classifier fot the NMIST handwritten digits database. It is recommended to use a convolution neural network.

# Project Team  
Qinyuan                 20137095  
Eamon Moloney           8457077  
Ibrahim Saana Aminu     25381993  
Des Powell              9513833  
Terence Coffey          15223124  

In [1]:
# ------------------------------------------------------
# Required imports
# ------------------------------------------------------
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.regularizers import l2
from keras.callbacks import EarlyStopping
import itertools
import numpy as np
from sklearn.metrics import confusion_matrix
import time
import matplotlib.pyplot as plt


# ------------------------------------------------------
# Constants
# ------------------------------------------------------
NUM_RUNS = 3
NUM_CLASSIFICATIONS = 10
INPUT_SHAPE = (28, 28, 1)
EPOCH_ITERATIONS = 30
VERBOSE_FLAG = 1
EXPLORE_MODE = 0

# ------------------------------------------------------
# Hyperparameter set of values
# ------------------------------------------------------
model_id_list = [3]
dropout_rate_list = [0.2]
batch_size_list = [32]
val_split_list = [0.2]
learn_rate_list = [0.0001]
kernel_size_list = [5]
filter_size_list = [15]
regularizer_list = [0.000001]
seed_value_list = [42]


# ------------------------------------------------------
# globals
# ------------------------------------------------------
experiment_count = 0
training_accuracy = {model_id: [] for model_id in model_id_list}
validation_accuracy = {model_id: [] for model_id in model_id_list}

# ------------------------------------------------------
# not being used - we always run EPOCH_ITERATIONS
# ------------------------------------------------------
early_stopping = EarlyStopping(monitor='val_loss', patience=4)
fit_callbacks = [early_stopping]


# ------------------------------------------------------
# create the specific set of models to be explored
# ------------------------------------------------------
def create_model(model_id, filter_size, kernel_size, dropout_rate, regularizer, seed_value):
    tf.random.set_seed(seed_value)

    # Define model structures based on model_id
    if model_id == 1:
        model = keras.Sequential([
            layers.Input(shape=INPUT_SHAPE),
            layers.Conv2D(filter_size, kernel_size=(kernel_size, kernel_size), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(filter_size * 2, kernel_size=(kernel_size, kernel_size), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dropout(dropout_rate),
            layers.Dense(10, activation="softmax")
        ], name='Model1')
        model_description = "Conv2D with two Conv layers, two MaxPool, Dropout, and Dense output."
    
    elif model_id == 2:
        model = keras.Sequential([
            layers.Input(shape=INPUT_SHAPE),
            layers.Conv2D(filter_size, kernel_size=(kernel_size, kernel_size), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Dropout(dropout_rate), 
            layers.Conv2D(filter_size * 2, kernel_size=(kernel_size, kernel_size), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Dropout(dropout_rate), 
            layers.Flatten(),
            layers.Dense(filter_size * 4, activation="relu"),
            layers.Dense(10, activation="softmax")
        ], name='Model2')
        model_description = "Conv2D with two of Conv,MaxPool,Dropout layers, with one internal Dense layer followed by Dense output."

    elif model_id == 3:
        model = keras.Sequential([
            layers.Input(shape=INPUT_SHAPE),
            layers.Conv2D(filter_size, kernel_size=(kernel_size, kernel_size), padding='same', activation="relu", 
                          kernel_initializer=HeNormal(seed=seed_value)),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(filter_size, kernel_size=(kernel_size, kernel_size), padding='same', activation="relu", 
                          kernel_initializer=HeNormal(seed=seed_value), 
                          kernel_regularizer=regularizers.l2(regularizer)),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dropout(dropout_rate),
            layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(regularizer)),   
            layers.Dropout(dropout_rate),
            layers.Dense(10, activation="softmax")
        ], name='Model3')
        model_description = "Structure based on Lecture notes detail"
    
    else:
        raise ValueError("Invalid model_id. Please use 1, 2, or 3.")
    
    return model, model_description


# ------------------------------------------------------
# Load and preprocess dataset
# ------------------------------------------------------
def load_and_preprocess_data():

    # Load and pre-process MNIST dataset
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0
    y_train = keras.utils.to_categorical(y_train, num_classes=NUM_CLASSIFICATIONS)
    y_test = keras.utils.to_categorical(y_test, num_classes=NUM_CLASSIFICATIONS)
    
    return (x_train, y_train), (x_test, y_test)


# ------------------------------------------------------
# Compile and train the model
# ------------------------------------------------------
def compile_and_train_model(model, x_train, y_train, batch_size, learn_rate, val_split):
    # compile the model
    model.compile(
        loss="categorical_crossentropy",
        optimizer=Adam(learning_rate=learn_rate),
        metrics=['accuracy', Precision(), Recall()]
    )
    # train the model
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=EPOCH_ITERATIONS, validation_split=val_split,shuffle=True,verbose=VERBOSE_FLAG) #callbacks=fit_callbacks)     
    
    return history


# ------------------------------------------------------
# Plot Training/Validation Accuracy 
# ------------------------------------------------------
def plot_training_metrics():
    nrows = len(model_id_list)
    ncols = NUM_RUNS
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols * 5, nrows * 4))
    fig.suptitle('Train and Validation Accuracy Across Models and Runs', fontweight='bold')
    colors = plt.cm.tab10.colors

    for row, model_id in enumerate(model_id_list):
        # Check if each model has collected data for NUM_RUNS
        if len(training_accuracy[model_id]) < NUM_RUNS:
            print(f"Warning: Model {model_id} has incomplete data for runs. Expected {NUM_RUNS}, got {len(training_accuracy[model_id])}")
            continue
        
        for col in range(NUM_RUNS):
            ax = axes[row, col] if nrows > 1 else axes[col]
            try:
                acc = training_accuracy[model_id][col]
                val_acc = validation_accuracy[model_id][col]
                color = colors[col % len(colors)]
                
                ax.plot(acc, color=color, linestyle='-', label='Train Accuracy')
                ax.plot(val_acc, color=color, linestyle='--', label='Val Accuracy')
                ax.set_title(f'Model {model_id} - Run {col + 1}')
                ax.set_xlabel('Epoch')
                ax.set_ylabel('Accuracy')
                ax.legend(loc='best')
            except IndexError:
                print(f"Error: Model {model_id} run {col + 1} data is missing.")
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()



# ------------------------------------------------------
# Run experiments
# ------------------------------------------------------
def run_experiments():
    (x_train, y_train), (x_test, y_test) = load_and_preprocess_data()
    
    global experiment_count
    for params in itertools.product(model_id_list,dropout_rate_list, batch_size_list, val_split_list, learn_rate_list, kernel_size_list, filter_size_list, regularizer_list, seed_value_list):
        model_id, dropout_rate, batch_size, val_split, learn_rate, kernel_size, filter_size, regularizer, seed_value = params  
        
        global training_accuracy, validation_accuracy
        experiment_count+=1
        
        # Create model with model description
        model, model_description = create_model(model_id,filter_size, kernel_size, dropout_rate, regularizer, seed_value)
        
        # Train and evaluate
        history = compile_and_train_model(model, x_train, y_train, batch_size, learn_rate, val_split)         
              
        # predict using the test data
        predictions = model.predict(x_test)
        predicted_labels = np.argmax(predictions, axis=1)
        true_labels = np.argmax(y_test, axis=1)
        conf_matrix = confusion_matrix(true_labels, predicted_labels)    
        
        # evaluate 
        results_model = model.evaluate(x_test, y_test, verbose=VERBOSE_FLAG)
        
        # log the data - just the accuracy detail for final delivery
        num_epochs_run = early_stopping.stopped_epoch if early_stopping.stopped_epoch > 0 else EPOCH_ITERATIONS
        log_header = ((f"#Epochs: {num_epochs_run}, Dropout Rate: {dropout_rate}, Batch Size: {batch_size}, Validation Split: {val_split}, "
            f"Learning Rate: {learn_rate}, Kernel Size: {kernel_size}, Filter Size: {filter_size}, Regularizer: {regularizer}, Seed: {seed_value}, "
            f"Description: {model_description}, "))
        log_data = ((f"Experiment #{experiment_count}, ACCURACY: Test Accuracy {results_model[1]:.4f}, Training Accuracy {history.history['accuracy'][-1]:.4f}, Validation Accuracy {history.history['val_accuracy'][-1]:.4f}, "
            f"(Test Loss {results_model[0]:.4f}, Test Precision {results_model[2]:.4f}, Test Recall {results_model[3]:.4f})"))
        data_to_log = (f"{model.name}" + ": " + log_data) if EXPLORE_MODE == 0 else (f"{model.name}" + ": " + log_header + log_data)
        print(data_to_log)
        
        # print out cm and 'model summary' detail
        print(f'Confusion Matrix\n{conf_matrix}\nModel Summary\n{model.summary()}')
        
        # gather accuracy numbers from plotting later
        training_accuracy[model_id].append(history.history['accuracy'])
        validation_accuracy[model_id].append(history.history['val_accuracy'])
        
               
                    
# ------------------------------------------------------
# main
# ------------------------------------------------------
# go for it...
experiment_count=0 # incremental counter that uniquely identifies the experiment
for number_of_runs in range(NUM_RUNS):
    start_time = time.time()
    print(f'Start Run #{number_of_runs+1} of {NUM_RUNS}')
    run_experiments()
    end_time = time.time()
    print(f'Finished Run #{number_of_runs+1} of {NUM_RUNS}, duration {(end_time-start_time):.4f} seconds')

# plot results from training
plot_training_metrics()

# the end




ModuleNotFoundError: No module named 'matplotlib'

Application using Keras for MNIST Digit Classifier using 2 convolution layers , 2 drop out layers and 2 accumulation layers. Split the dataset into training and test data in the ratio 70 percent and 30 percent respectively.

Display accuracy of Testing and Validation dataset

Discuss the Project especially
- How you choose to tackle it
- What design decisions you made
- What the results are like
- What you might do better/differently next time you had to tackle a similar project
- If plots are called for they should be in your code and in your report.
- Marks for neat well designed code with appropriate level of comments
- neat logically laid out and informative reports.
- Provide classification accuracy for the training and test data. The test data should be split in the ration 70 to 80 and the baance for validation.


